<h1>Data Driven Business: Data Science pipeline.</h1>

Om nuttige voorspellingen te doen, moet eerst bepaald worden wat voorspeld moet worden. Dit document in bedoelt om later in het hoofd-document in integreren, maar is voor nu stand-alone, en dekt de Business-Understanding, Data-Understanding en (gedeeltelijk) Data-Preparation voor het target variabele af.

<h2>Target variabele.</h2>

<h3>Business understanding</h3>

Door de interviews met onze business-expert hebben begrepen dat het voorspellen van moment van 'functieherstel' toegevoegde waarde voor ProRail zou hebben. Dit is het moment dat een traject weer klaar is om treinverkeer toe te staan. Via ProRail hebben wij dit diagram aangeleverd gekregen, wat inzicht geeft in het bedrijfsprocess:

![title](images/badkuip.png)
De X-as beslaat tijd, en de Y-as hoeveelheid treinverkeer (beiden non-proportioneel). Dit model zal naar gerefereerd worden als 'badkuip model'.

Functieherstel staat niet expliciet benoemd in dit model, maar vind plaats in de buurt van het "Storing-Eind" moment, en wanneer de lijn weer omhoog begint te bewegen op de Y-as.

ProRail baat bij een goede voorspelling wat betreft moment van functieherstel, zodat ze kunnen zorgen dat het opstarten van de dienstregeling zo goed mogelijk hierop kan aansluiten.

Uit de interviews blijkt dat de gegevens die waarschijnlijk tot een goede prognose leiden waarschijnlijk pas beschikbaar zijn wanneer een aannemer ter plekke is, in het badkuipmodel benoemd als "oplosteam aanwezig". Als het goed is zijn zowel het tijdstip van aankomst van de monteur, als het tijdstip van functieherstel aanwezig in de dataset, hoewel dit later in 'Data Understanding' bevestigd moet worden.

De verstreken tijd tussen de melding van een incident, en het moment dat een oplosteam aanwezig is is sterk variabel. Hoewel dit in de toekomst wellicht ook te voorspellen zou zijn met een model, beperken wij in onze huidige aanpak tot het voorspellen van 'aankomst aannemer' tot 'functieherstel', omdat dat volgens onze huidige business-kennis het best voorspelbaar is aan de hand van de attributen van een specifieke storing.

<h3>Data Understanding.</h3>

Om de mogelijkheid van het implementeren van het targetvariabele van (tijdstip functieherstel) - (tijdstip aankomst aannemer) te onderzoeken, moet de data geïnspecteerd om te bepalen of dit ook echt realistisch is. Hiervoor onderzoeken we de eerder benoemde hypothese voor targetvariabele, en eventuele variabelen die hier concurrentie voor zouden kunnen zijn, of meer inzicht zouden kunnen geven.

We beginnen met het importeren van een aantal libraries, en de dataset:

In [ ]:
import enum

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn import dummy
from sklearn import linear_model
from sklearn import model_selection
from sklearn import tree
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression


In [ ]:
df = pd.read_csv("sap_storing_data_hu_project.csv", low_memory=True)
df

Omdat de dataset redelijk groot is, kosten veel van de operaties die tijdens het verkennen van de data gebeuren, erg veel tijd. Volgens onze business-expert zou een willikeurige selectie van ongeveer 10% van de dataset alsnog representatief voor de hele dataset moeten zijn, dus voor het verkennen maken we dit meteen aan.

In [ ]:
sample = df.sample(frac=0.1, random_state=1)

Iets interessants om te onderzoeken is de relatie tussen de tijdstippen van functieherstel, storing eind, en tijdstip van melding. We maken voor de onderzoek doeleinden hier aparte variabelen van voor gemak, en kijken naar de datatypes.

In [ ]:
fh_tijdstip = sample["stm_fh_ddt"]
stor_eind = sample["stm_sap_storeind_ddt"]
stm_sap_meld_ddt = sample["stm_sap_meld_ddt"]

In [ ]:
print(f"""{fh_tijdstip.dtype}
{stor_eind.dtype}
{stm_sap_meld_ddt.dtype}""")

Al deze kolommen bestaan uit Strings. Deze moeten omgezet worden naar DateTime voor effectief rekenwerk. Hierna slaan we ze op in een apart DataFrame voor onderzoek. Uit dit dataframe verwijderen we alle rijen waar minimaal 1 NaN waarde instaat, omdat dit dataframe bedoelt is om de werking tussen de verschillende kolommen te vergelijken.

Ook definieren we een TimeDelta object voor 2 minuten, omdat het exact vergelijken van tijdstippen voor onze doelen vaak niet nuttig is, en als dingen binnen 2 minuten van elkaar vallen, dit in veel gevallen meer informatie kan geven.

In [ ]:
fh_tijdstip = pd.to_datetime(fh_tijdstip, errors="coerce")
stor_eind = pd.to_datetime(stor_eind, errors="coerce")
stm_sap_meld_ddt = pd.to_datetime(stm_sap_meld_ddt, errors="coerce")

twominutes = pd.to_timedelta(2, unit="m")

In [ ]:
#sample_de_1 = DataFrame DataExploration 1
df_de_1 = pd.DataFrame({"fh_ddt": fh_tijdstip,
                       "se_ddt": stor_eind,
                       "meld_ddt": stm_sap_meld_ddt}).dropna()

df_de_1.shape  # 1/3rd is dropped from nas

Een eerste vraag om te onderzoek is naar het verschil tussen "Storing eind" en "Functieherstel". Uit ons business-onderzoek begrijpen we dat deze waarschijnlijk de volgende betekenissen hebben:

Functieherstel: Het moment dat er theoretisch weer treinverkeer over het traject kan.
Storing eind: Wanneer het probleem opgelost is, en de herstelwerkzaamheden voltooid zijn.

Omdat het probleem vaak opgelost is, wanneer er weer treinen over het traject kunnen, zullen deze 2 momenten vaak tegelijk zijn. Dit kunnen we ook bevestigen:

In [ ]:
def percentage(numerator: float, denominator: float) -> float:
    """Calculate a percentage from a fraction.

    Args:
        numerator: fraction numerator.
        denominator: fraction denominator.

    Returns:
        Percentage between 0 and 100, as float."""
    return (numerator / denominator) * 100


n_fh_is_se = (np.abs(df_de_1["fh_ddt"] - df_de_1["se_ddt"]) < twominutes).sum()
f"Storing Eind is in {percentage(n_fh_is_se, df_de_1.shape[0])}% van de gevallen binnen 2 minuten van Functie Herstel."

In veel andere gevallen zou men verwachten dat storing eind na functieherstel is, wanneer de aannemer het spoor weer operationeel krijgt, maar daarna wel nog werk moet afmaken.

In [ ]:
n_fh_voor_se = ((df_de_1["se_ddt"] - twominutes) > df_de_1["fh_ddt"]).sum()
f"Storing eind is in {percentage(n_fh_voor_se, df_de_1.shape[0])}% van de gevallen meer dan 2 minuten na functieherstel."

Dit correspondeerd met de verwachting van Business Understanding.

Eerder hebben wij op basis van businesskennis een belovend targetvariabele vastgesteld, namelijk het verschil in tijd tussen het arriveren van de aannemer, en functieherstel. Om dit daadwerkelijk te kunnen gebruiken moet de daadwerkelijke data die hieraan correspondeerd geïnspecteerd worden.

In de data dictionairy staan specifiek een kolom voor datum en tijd (apart) benoemd voor het arriveren van de aannemer, namelijk "stm_aanntpl_dd" voor datum en "stm_aanntpl_tijd" voor tijd. In de dataset zit ook een kolom "stm_aanntpl_ddt", hoewel deze in de dictionairy als n.v.t. bestempeld is. Toch lijkt het nuttig het verschil tussen deze kolommen even te inspecteren, om te kijken welke van deze het beste is.

Ten eerste is het handig een beeld te hebben hoeveel NaN waardes er in deze kolommen zitten.

In [ ]:
n_nan_aanntpl_dd = sample["stm_aanntpl_dd"].isna().sum()
f"De datumkolom bestaat voor {percentage(n_nan_aanntpl_dd, sample.shape[0])}% uit NaN waardes."

In [ ]:
n_nan_aanntpl_tijd = sample["stm_aanntpl_tijd"].isna().sum()
f"De tijdkolom bestaat voor {percentage(n_nan_aanntpl_tijd, sample.shape[0])}% uit NaN waardes."

In [ ]:
n_nan_aanntpl_ddt = sample["stm_aanntpl_ddt"].isna().sum()
f"De gecombineerde datum en tijdkolom bestaat voor {percentage(n_nan_aanntpl_dd, sample.shape[0])}% uit NaN waardes."

De hoeveelheid NaN waardes komt overeen tussen de dd en ddt kolommen. Om te bevestigen dat deze gelijk zijn kunnen we de datum en tijd kolommen combineren naar één, en ze allemaal als DateTime objecten in een apart DataFrame zetten.

In [ ]:
ddt_format = sample["stm_aanntpl_dd"] + " " + sample["stm_aanntpl_tijd"]

df_aan = pd.DataFrame({"t": pd.to_datetime(sample["stm_aanntpl_tijd"], errors="coerce"),
                       "d": pd.to_datetime(sample["stm_aanntpl_dd"], errors="coerce"),
                       "dt": pd.to_datetime(sample["stm_aanntpl_ddt"], errors="coerce"),
                       "dt_own": pd.to_datetime(ddt_format, errors="coerce")}).dropna()


In [ ]:
f"In {percentage((df_aan['dt'] == df_aan['dt_own']).sum(), df_aan.shape[0])}% van de non-NaN gevallen is de ddt kolom gelijk aan aan de datum en tijd kolom."

Al met al kunnen we dus net zo goed de ddt_tabel in de originele dataset gebruiken als targetvariabele.

Dan nog even kijken of er idd genoeg nuttige entries zijn voor functieherstel - arrival monteur

In [ ]:
possible_target = df_de_1["fh_ddt"] - df_aan["dt"]  # Tijdstip functieherstel - tijdstip aankomst aannemer.
f"De mogelijke target bestaat voor {percentage(possible_target.isna().sum(), possible_target.shape[0])}% uit NaN-waardes."

In [ ]:
possible_target.dropna(inplace=True)
f"Wanneer deze NaN-waardes worden weggelaten hou je van de originele dataset uiteindelijk ongeveer {percentage(possible_target.shape[0], sample.shape[0])}% over die een targetvariabele hebben."

Om een nuttig targetvariabele te creeëren, zouden we uiteindelijk alle TimeDelta objecten die we gecreeërd hebben omzetten naar de hoeveelheid seconden waaruit dit tijdsinterval bestaat. Hoewel rekenen in seconden het inzicht mogelijk moeilijker maakt, verhoogt dit wel de granulariteit van het targetvariabele, waardoor deze zich meer gedraagd als continue waarde, wat de performance van het model weer ten goede komt. Om inzicht te verbeteren in performance, definieren we voor het gemak wel even een functie die een seconden-series naar minuten omzet, zodat grafieken leesbaarder gemaakt kunnen worden.

In [ ]:
class TimeUnit(enum.Enum):
    """Enum that represents a unit of measurement of time, to enable easy conversion from one unit to another.

    Values for units of measurements are measured in seconds."""
    second = 1
    minute = 60
    hour = 60 * 60
    day_night_cycle = 60 * 60 * 24


def convert_timeunit(value: pd.Series or float or int,
                     from_type: TimeUnit,
                     to_type: TimeUnit) -> float:
    """Convert an amount of time as number, or a series of those, to another.

    Not class attribute of TimeUnit class to allow for better typing.

    Args:
        value:
            a number, or a series of numbers, that represent an amount of time in a certain unit of measurement as defined in
            TimeUnit.
        from_type: TimeUnit scale of measurement of time that input is in.
        to_type: TimeUnit scale of measurement of time that output should be in.

    Returns:
        Time measurement(s) converted to new unit of measurement.
        Scalar if input was scalar, array/series of input type if input was array/series."""
    coef = from_type.value / to_type.value
    return value * coef

We creeëren voor nu een apart variabele voor het targetvariabele als seconden. Deze bevat mogelijk een aantal 0-waarden, laten we deze eerst inspecteren.

In [ ]:
de_target_secs = possible_target.dt.total_seconds()
f"Het targetvariabele bestaat voor {percentage((de_target_secs == 0).sum(), de_target_secs.shape[0])}% uit 0 seconden."

Het is wellicht interessant een beeld te hebben van outliers van ons targetvariabele:

In [ ]:
plt.boxplot(de_target_secs)

Uit deze bijna-onleesbare boxplot blijkt dat vooral boven de bovengrens, er heel veel extreme outliers zijn. Aan de onderkant zijn er ook een aantal negatieve waarden. Als we er vanuit gaan dat dit niet door foutive data word veroorzaakt, betekend dit waarschijnlijk dat er al weer treinen kunnen rijden, voordat er uberhaupt een monteur aan te pas komt. Hier moet wat aan gedaan worden.

Een standaardmethode om outliers te verwijderen, is de interkwartielafstand(<i>IQR</i>) berekenen, en alle waarnemingen 3IQR boven het 3e kwartiel, en 3IQR onder het eerste kwartiel te verwijderen. Het lijkt ons echter verstandiger in dit geval het verwijderen van outliers te baseren op business-kennis.

Om een nuttig voorspellend model te maken, is het waarschijnlijk handig om een ondergrens in functiehersteltijd te stellen vanaf het moment dat de aannemer komt. Omdat wij inschatten dat bij storingen onder de 5 minuten de aannemer zelf ook kan inschatten dat het heel kort gaat duren, en een voorspelling hier waarschijnlijk niet nuttig is, laten we deze weg uit onze dataset.

Onze business-expert heeft aangegeven alle hersteltijden boven de 6u niet interessant te vinden, dus deze waarnemingen laten we als NaN waarde in ons targetvariabele, en maken we een aparte binaire-categorie kolom voor, zodat dit eventueel in de toekomst nog gebruikt kan worden voor een apart model.

Om de verdelingen voor het eerste model te kunnen onderzoeken, halen we bij de data-exploration set nu alles onder 5 minuten en boven 6 uur eruit.



In [ ]:
de_target_secs = de_target_secs.loc[(de_target_secs < convert_timeunit(6, TimeUnit.hour, TimeUnit.second)) & (de_target_secs > convert_timeunit(5, TimeUnit.minute, TimeUnit.second))]

In [ ]:
(convert_timeunit(de_target_secs, TimeUnit.second, TimeUnit.hour)).hist(bins=50)

Ons targetvariabele lijkt exponentieel verdeeld te zijn.

Hiernaast definiëren we wel een functie die outliers op de klassieke wijze verwijderd, voor gebruik later.

In [ ]:
def remove_outliers(s: pd.Series, fence: float = 3) -> pd.Series:
    """Remove the outliers from a Pandas series by removing all values that lie below q1 or above q3 with
    the product of the inter quartile distance and the 'fence'.

    Args:
        s: Series to remove outliers from.
        fence: amount of IQRs value should lie above q3 or below q1 with to get designated an outlier.

    Returns:
        Series without outliers."""
    q1, q3 = s.quantile(.25), s.quantile(.75)
    iqr = q3 - q1
    fence_iqr_prod = fence * iqr
    return s.loc[(s > q1 - fence_iqr_prod) & (s < q3 + fence_iqr_prod)]

<h3>Data preperation.</h3>

Nu we een idee hebben van een (hopelijk) bruikbaar target variabele, kunnen de benodigde operaties op de originele dataset uitgevoerd worden, om hieruit een volledig uitgewerkt targetvariabele voor later gebruik uit te creeëren.

We beginnen door de 2 benodigde kolommen voor het target variabele naar DateTime objecten te veranderen in nieuwe kolommen.

In [ ]:
df["stm_aanntpl_ddt_as_pddt"] = pd.to_datetime(df["stm_aanntpl_ddt"], errors="coerce")
df["stm_fh_ddt_as_pddt"] = pd.to_datetime(df["stm_fh_ddt"], errors="coerce")

Hierna maken we een aparte kolom in de dataframe van het tijdstip van functieherstel, min tijdstip van arriveren van de monteur.

In [ ]:
df["fh_min_aanntpl"] = df["stm_fh_ddt_as_pddt"] - df["stm_aanntpl_ddt_as_pddt"]

We slaan van deze nieuwe kolom het verschil op als seconden in een apart variabele.

In [ ]:
target = df["fh_min_aanntpl"].dt.total_seconds()

We laten hieruit alles boven de 6 uur, en alles onder de 5 minuten weg, en slaan dit op in een nieuwe variabele.

In [ ]:
boven_5_min = target > convert_timeunit(5, TimeUnit.minute, TimeUnit.second)
onder_6_uur = target < convert_timeunit(6, TimeUnit.hour, TimeUnit.second)

target = target.loc[boven_5_min & onder_6_uur]  # Gemeten in seconden.

target.head(5)

<h3>Modelling</h3>

Met een volledig voorbereid targetvariabele, kan er een 'BaseLine' model gemaakt worden, om een RMSE score vast te stellen die overtrefd kan worden. De meest logische strategie hiervoor is het gemiddelde gokken van alle bekende waardes. Hiervoor gebruiken we van SciKitLearn de DummyRegressor, met de 'mean' strategie.

In [ ]:
baseline = dummy.DummyRegressor(strategy="mean")
baseline_X = np.arange(target.shape[0])  # Dummy feature variabelen voor BaseLine.
baseline.fit(np.arange(target.shape[0]), target)

Van dit baseline model kunnen we vervolgens de RMSE score in seconden bepalen.

In [ ]:
baseline_rmse = metrics.mean_squared_error(target, baseline.predict(baseline_X), squared=False)
baseline_rmse

Voor inzicht is het ook wel interessant deze score even in minuten te bekijken, omdat dit toch meer inzicht geeft in de business-context.

In [ ]:
convert_timeunit(baseline_rmse, TimeUnit.second, TimeUnit.minute)

Dit model slaan we op in een DataFrame, waar later ook andere modellen in opgeslagen kunnen worden, zodat deze overzichtelijk bekeken kunnen worden.

In [ ]:
models = pd.DataFrame({"Title": ["BaseLine model with mean strategy"],
                       "Model": [baseline],
                       "RMSE": [baseline_rmse]})

models

<h2>Feature variabelen: model 1</h2>

<h2>Business understanding</h2>

Uit onze interviews met ProRail blijken dat een aantal variabelen die waarschijnlijk in de Dataset te vinden zijn, van significante invloed zijn op ons targetvariabele. Dit zijn:

<ul>
<li>Type storing</li>
<li>Tijdstip op de dag</li>
<li>Dag in het jaar</li>
<li>Inschatting treinverkeersleider</li>
<li>Inschatting aannemer</li>
<li>Monitoringscode</li>
<li>Gebied</li>
</ul>

Met als doel deze variabelen uiteindelijk in een model te gebruiken. We zullen van het CRISP-DM process per variabele de Data-Understanding en Data-Preperation stappen behandelen, met waar relevant Business-Understanding.
Per featurevariabele:

<h3>Monitorings informatie</h3>

Zoals eerder in het document is genoemd is, zou informatie vanuit de monitoring nuttig kunnen zijn voor een voorspellend model.

Hier gaan we dit onderzoeken.

Vanuit business understanding weten we dat alleen de monitorings code relevant is, dus we gaan hier onderzoeken welke kolommen dat zijn.


In de data dictionairy hebben 2 kolommen gevonden die mogelijk relevant zijn, namelijk: 'stm_scenario_mon' & 'stm_mon_nr__statuscode'.

We zullen nu kijken hoeveel NA's deze kolommen bevatten.

In [ ]:
df[['stm_mon_nr__statuscode', 'stm_scenario_mon']].isna().sum()

Aantal NA's per kolom:

<ul>
<li>stm_mon_nr__statuscode   = 571685</li>
<li>stm_scenario_mon         = 862111</li>
</ul>

Hier uit kunnen we concluderen dat de kolom 'stm_scenario_mon' niet bruikbaar is. Deze kolom bevat simpelweg te veel NA's om mee te werken.

We zullen nog wel verder onderzoek doen op de kolom 'stm_mon_nr__statuscode'.

We zullen nu kijken naar het aantal unieke waarde in de kolom.

In [ ]:
df['stm_mon_nr__statuscode'].nunique()

Nu weten we dat de kolom 9 unieke waardes bevat.  (10 als je NAN zou includen)

Dit is een fijn aantal om mee te werken.

Nu zullen we een boxplot maken waarin de verschillende monitoring statuscodes worden geplot tegenover het target variabele.

Dit zal ons inzicht geven in het invloed per category van monitoring statuscode.

In [ ]:
mon_code_col = np.array(["stm_mon_nr__statuscode"])
mon_code = sample[mon_code_col].astype("category")
mon_code["target"] = target

sns.boxplot(x=mon_code["stm_mon_nr__statuscode"], y=mon_code["target"])

In deze boxplot is te zien dat de mediaan van alle categorieen relatief dichtbij elkaar zitten. Van de hoogste tot laagste mediaan is het verschil maximaal rond de 1000 seconden.

Voor nu concluderen wij dat deze data niet genoeg invloed zal hebben op onze target. Mogelijk komen we hier later nog op terug als ons model meer data vereist.

<h3>Tijdst op de dag, en dag in het jaar.</h3>

Hier gaan we de data uit kolom 'stm_sap_meld_ddt' bruikbaar maken voor toekomstige modellen.
Dit doen we door de datetime om te zetten naar zowel: aantal secondes vanaf middernacht & dag in het jaar

Als eerst converten we stm_sap_meld_ddt naar een pandas datetime datatype. Deze kolom krijgt de naam: 'stm_sap_meld_ddt_as_pddt'.
Op deze manier hebben we de originele data, en de omgezette data beschikbaar.

In [ ]:
df['stm_sap_meld_ddt_as_pddt'] = pd.to_datetime(df["stm_sap_meld_ddt"], errors="coerce")

Nu we een kolom hebben die in het pandas datetime datatype staat, kunnen we er gemakkelijk mee werken.

We gaan hier 2 kolomen aan df toevoegen, namelijk:
- stm_sap_meld_sec_mn:  Hierin staat het aantal secondes vanaf  00:00  tot het moment van de melding
- stm_sap_meld_day_count:  Hierin staat de hoeveelste dag in dat jaar het is, op het moment van de melding

In [ ]:
df['stm_sap_meld_sec_mn'] = (df['stm_sap_meld_ddt_as_pddt'] - df['stm_sap_meld_ddt_as_pddt'].dt.normalize()) / pd.Timedelta(seconds=1)
df['stm_sap_meld_day_count'] = df["stm_sap_meld_ddt_as_pddt"].dt.day_of_year

Deze kunnen samen met het targetvariabele gecombineerd worden in één DataFrame voor gebruik in model 1.

In [ ]:
m1_df = pd.DataFrame({"Target": target,
                      "Tijdstip": df["stm_sap_meld_sec_mn"],
                      "Dag": df["stm_sap_meld_day_count"]})

<h2>Type storing</h2>

<h3>Data exploration</h3>

Uit ons business-onderzoek verwachten we dat het type storing waar een verlies van functie door veroorzaakt word is is van grote invloed op de tijd die het kost om functieherstel te bereiken. Attributen uit de dataset die hier wellicht invloed op kunnen hebben zijn:
<ol>
<li>stm_oorz_code: de oorzaak-code</li>
<li>stm_oorz_groep: de oorzaak-groep</li>
<li>stm_techn_mld: het techniekveld van de melding</li>
</ol>

We zullen deze kolommen onderzoeken, tot we er een vinden die relevant lijkt te zijn.

We maken van deze 3 kolommen uit de 10% subset van de dataset, en de target.

In [ ]:
de_type_cols = np.array(["stm_oorz_code", "stm_oorz_groep", "stm_techn_mld"])

de_type = sample[de_type_cols].astype("category")
de_type["target"] = target
de_type.head(5)

Het is handig te weten hoeveel permutaties elk variabele heeft.

In [ ]:
de_type[de_type_cols].nunique()

Om de curse of dimensionality te voorkomen is het handig zo weinig mogelijk dimensies te introduceren met het encoderen van deze categoriale variabelen. Daarom kan het handig zijn bij zo een kolom te blijven die zo weinig mogelijk permutaties heeft. We zullen deze attributen van weinig naar veel onderzoeken.

We beginnen dus bij stm_oorz_groep. Het is interessant om te weten hoe de verschillen in verdeling van het targetvariabele zijn per kolom. Hiervoor kunnen we boxplots maken per category.

In [ ]:
sns.boxplot(x=de_type["stm_oorz_groep"], y=de_type["target"])

De oorzaak groep lijkt niet veel te zeggen over het targetvariabele.

In [ ]:
sns.boxplot(x=de_type["stm_techn_mld"], y=de_type["target"])

Hier lijken we al meer mee te kunnen. We dummyencoden deze en voegen hem toe aan het dataframe voor model één.

In [ ]:
type_encoded = pd.get_dummies(df["stm_techn_mld"], prefix="Techniekveld")
type_encoded.head(1)

type_encoded.head()

In [ ]:
m1_df = m1_df.merge(type_encoded, left_index=True, right_index=True)
m1_df

<h2>Prognose aannemer.</h2>

Hoewel uit de interviews blijkt dat de prognose van de aannemer over de duur van functieherstel waarschijnlijk pessimistisch is, zal deze waarschijnlijk toch wel veel zeggen over de daadwerkelijke duur van functieherstel. De aannemer maakt de eerste prognose vrij snel nadat die aankomt, waardoor deze dus ook in het model meegenomen kan worden.

<h3>Data understanding</h3>

In de dataset is een kolom "stm_progfh_in_duur", die in de data dictionairy benoemd staat als "?Duur FHT". Deze kolom is waarschijnlijk een mogelijk attribuut voor de voorspelde duur van functieherstel. Deze gaan we eerst onderzoeken. Ten eerste is een klein inzicht in de data, en het datatype handig.

In [ ]:
sample["stm_progfh_in_duur"].head(5)

In [ ]:
sample["stm_progfh_in_duur"].dtypes

Deze kolom word als 'O' geïntrepeteerd door Pandas, hoewel de waardes numeriek lijken. We kunnen deze omzetten naar proper numieriek datatype. Het is interessant om te weten met hoeveel waardes we hierna over blijven die Pandas niet kan intrepeteren, en dus NaN-waardes van maakt.

In [ ]:
de_prog_duur = pd.to_numeric(sample["stm_progfh_in_duur"], errors="coerce")
f"De stm_progfh_in_duur kolom bestaat voor {percentage(de_prog_duur.isna().sum(), de_prog_duur.shape[0])}% uit NaN-waardes."

Een redelijke hoeveelheid. Om de verdeling te bekijken kunnen we NaN-waardes veilig verwijderen.

In [ ]:
de_prog_duur = de_prog_duur.dropna()
de_prog_duur.head(5)

In [ ]:
de_prog_duur.hist(bins=50)

Het histogram bij dit variabele is redelijk opvallend. Het inspecteren van de data bleek eerder uit dat deze veel numerieke waardes boven de 1 bevat. Toch zitten deze niet in het histogram. Mogelijk word het creeeren van het histogram verstoord door veel extreme hoge/lage waardes. Dit is wel interessant om te onderzoeken.

In [ ]:
f"De stm_progfh_in_duur kolom bestaat voor {percentage((de_prog_duur == 0.0).sum(), de_prog_duur.shape[0])}% uit 0.0."

Ook leek er een absurd hoge waarde te zijn die vaak ingevuld is, als onechte waarde. Deze kunnen we ook onderzoeken.

In [ ]:
f"De stm_progfh_in_duur kolom bestaat voor {percentage((de_prog_duur == 99999999.0).sum(), de_prog_duur.shape[0])}% uit de waarde 99999999."

De betekenis van de prognoseduur is ook niet helemaal duidelijk uit de gegeven informatie. Is deze duur van het moment dat de aannemer aankomt, en deze prognose maakt, tot het moment van functieherstel? Of dit vanaf het moment dat de storing begint? Om dit te onderzoeken hebben we ook de datum en het tijdstip waarop de prognose van functieherstel valt in de dataset. Hier kunnen we een apart variabele van maken.

In [ ]:
de_prog_moment = sample["stm_progfh_in_datum"] + " " + sample["stm_progfh_in_tijd"]
de_prog_moment = pd.to_datetime(de_prog_moment, errors="coerce")
de_prog_moment.head(2)

Om vast te stellen of dit variabele inderdaad gelijk is aan dezelfde periode die we afdekken met ons targetvariabele, kunnen we het moment van prognose nemen, en hier het moment dat de aannemer aankomt van aftrekken. We kunnen volgens uit de dataset ook het tijdstip van aankomen van de aannemer halen, en hier vervolgens een variabele voor prognoseduur vanaf het moment dat de aannemer aankomt van maken.

In [ ]:
de_prog_aankomst_aann = pd.to_datetime(sample["stm_aanntpl_ddt"], errors="coerce")
de_prog_aankomst_aann.shape

In [ ]:
de_prog_duur_2 = (de_prog_moment - de_prog_aankomst_aann).dt.total_seconds()

In [ ]:
de_prog_duur_2.shape

Van de_prog_duur_2 weten we dat dit in seconden is gemeten. Van de_prog_duur weten we dat niet zeker. Het is wel interessant om te kijken of deze 2 gelijk aan elkaar zijn, omdat de_prog_duur anders waarschijnlijk niet het beste featurevariabele is wat betreft businessdoelstelling. We nemen een foutmarge van 2 minuten met het vergelijken van de 2 tabellen, want als dit verschil er tussen zit, zouden ze nog steeds hetzelfde kunnen betekenen, met kleine notatiefouten.

In [ ]:
de_prog_gelijk_in_seconden = ((np.abs(de_prog_duur_2 - de_prog_duur)) < 120).sum()
f"Wanneer je de tabellen als seconden behandeld, is {percentage(de_prog_gelijk_in_seconden, de_prog_duur_2.shape[0])}% van de kolommen (bijna) gelijk."

Dit is duidelijk niet genoeg, dus we kunnen ook kijken of de prognose duur kolom in minuten is gemeten, en dan dezelfde vergelijking doen.

In [ ]:
de_prog_duur_2_min = convert_timeunit(de_prog_duur_2, TimeUnit.second, TimeUnit.minute)

de_prog_gelijk_in_minuten = ((np.abs(de_prog_duur_2_min - de_prog_duur)) < 2).sum()
f"Wanneer je de tabellen als minuten behandeld, is {percentage(de_prog_gelijk_in_minuten, de_prog_duur_2.shape[0])}% van de kolommen (bijna) gelijk."

Hoewel zonder de NaN waardes, dit inderdaad de betekenis van de kolom lijkt te zijn, is de veiligere aanpak die meer correcte waardes in stand houd, de zelf gecreeërde kolom.

We kunnen nu ook naar de waardes in deze kolom kijken. Hiervoor verwijderen we de NaN waardes.

In [ ]:
de_prog_duur_2 = de_prog_duur_2.dropna()
de_prog_duur_2.shape

Om meer inzicht in deze gegevens te krijgen, is het ook noodzakelijk de outliers te verwijderen. Hiervoor gebruiken we dezelfde logica als bij het targetvariabele.

In [ ]:
(de_prog_duur_2 < convert_timeunit(6, TimeUnit.hour, TimeUnit.second)).sum()

In [ ]:
(de_prog_duur_2 > convert_timeunit(5, TimeUnit.minute, TimeUnit.second)).sum()

In [ ]:
boven_5_min = de_prog_duur_2 > convert_timeunit(5, TimeUnit.minute, TimeUnit.second)
onder_6_uur = de_prog_duur_2 < convert_timeunit(6, TimeUnit.hour, TimeUnit.second)

de_prog_duur_2 = de_prog_duur_2.loc[boven_5_min & onder_6_uur]  # Gemeten in seconden.

de_prog_duur_2.shape

In [ ]:
de_prog_duur_2.hist(bins=20)

Dit variabele waarschijnlijk heeft een redelijk gelijke exponentiële verdeling als ons targetvariabele. Zouden deze dan ook correleren?

In [ ]:
de_prog_duur_2.shape

In [ ]:
de_prog_corr_matrix = pd.DataFrame({"Target": target,
                                    "Prognose": de_prog_duur_2}).dropna()
de_prog_corr_matrix.corr()

Deze 2 variabele correleren sterk. Dit is dan visueel waarschijnlijk ook te zien.

In [ ]:
plt.scatter(de_prog_corr_matrix["Target"], de_prog_corr_matrix["Prognose"], s=1, alpha=0.5)
plt.xlabel("Target")
plt.ylabel("Prognose")
plt.title("Prognose tegenover target.")

Hiermee concluderen we dat dit een goed featurevariabele is, welke goed in een model gebruikt kan worden.

<h3>Data preperation</h3>

Nu dit featurevariabele is vastgesteld, blijft er alleen over deze goed te engineeren voor de hele dataset, en dit aan het dataframe voor het model toe te voegen.

In [ ]:
progfh_formatted = df["stm_progfh_in_datum"] + " " + df["stm_progfh_in_tijd"]
df["stm_progfh_as_pddt"] = pd.to_datetime(progfh_formatted, errors="coerce")

In [ ]:
prognose = (df["stm_progfh_as_pddt"] - df["stm_aanntpl_ddt_as_pddt"]).dt.total_seconds()
m1_df.head(2)

In [ ]:
prognose.shape

In [ ]:
boven_5_min = prognose > convert_timeunit(5, TimeUnit.minute, TimeUnit.second)
onder_6_uur = prognose < convert_timeunit(6, TimeUnit.hour, TimeUnit.second)

prognose = prognose.loc[boven_5_min & onder_6_uur]  # Gemeten in seconden.

target.head(5)

In [ ]:
prognose.shape

In [ ]:
m1_df["Prognose"] = prognose

In [ ]:
m1_df["Prognose"].hist(bins=20)

<h2>Locatie storing</h2>

<h3>Data exploration</h2>

Uit ons businessonderzoek blijkt dat een idee van waar een storing gebeurt ook iets kan zeggen over de duur tot functieherstel. Een aantal kolommen zouden hier mogelijk voor gebruikt kunnen worden. Bijvoorbeeld "stm_geo_mld", "stm_contractgeb_mld" of "stm_geo_gst". Omdat één gebied niet persé volgordelijk zijn, kunnen we deze variabelen als nominaal beschouwen. Omdat een variabele met heel veel permutaties waarschijnlijk tot te veel dimensies voor nuttige modellen zou lijden, is het handig een variabele te kiezen met niet al te veel permutaties. Hier kunnen we naar kijken.

In [ ]:
df[["stm_geo_mld", "stm_geo_gst", "stm_contractgeb_mld"]].nunique()

stm_contractgeb_mld heeft niet al te veel permutaties. Ook hebben we van ProRail hier de betekenissen van gekregen.

In [ ]:
sample["stm_contractgeb_mld"].head(5)

In [ ]:
contractgebied_betekenissen = pd.read_excel("Contractgebiedcodes.xlsx")
contractgebied_betekenissen

Elk van deze categoriën staat voor een bepaald contractgebied, bijvoorbeeld "Utrecht" of "overig". Uit de betekenissen weten we dat veel van deze gebiedscodes voor dezelfde gebieden staan.

<h3>Data preperation.</h3>

Om de permutaties te verminderen kunnen we met een replace operatie alle duplicaatbetekenissen naar 1 categorie veranderen. We kunnen alle codes met dezelfde betekenis naar één code veranderen.

In [ ]:
contractgebied = df["stm_contractgeb_mld"]
contractgebied = contractgebied.replace({81.0: 1.0,
                                         71.0: 4.0,
                                         7.0: 6.0,
                                         11.0: 10.0,
                                         16.0: 12.0,
                                         14.0: 13.0,
                                         15.0: 13.0,
                                         18.0: 19.0,
                                         20.0: 23.0,
                                         21.0: 22.0,
                                         24.0: 27.0,
                                         28.0: 30.0,
                                         29.0: 31.0,
                                         33.0: 31.0,
                                         34.0: 35.0,
                                         36.0: 37.0,
                                         63.0: 0.0,
                                         61.0: 0.0,
                                         51.0: 0.0,
                                         52.0: 0.0,
                                         60.0: 0.0,
                                         58.0: 0.0,
                                         59.0: 0.0,
                                         70.0: 0.0,
                                         25.0: 0.0,
                                         26.0: 0.0})


En voor het volledige kan dit ook nog even naar een categorie datatype veranderd worden.

In [ ]:
contractgebied = contractgebied.astype("category")

We kunnen dit met een dummyencoding bruikbaar maken voor een model.

In [ ]:
loc_encoded = pd.get_dummies(contractgebied, prefix="Contractgebied")

En dit kan vervolgens toegevoegd worden aan het dataframe voor gebruik in een model.

In [ ]:
m1_df = m1_df.merge(loc_encoded, left_index=True, right_index=True)

<h2> Dingen voor modelling, maak mooier!</h2>

In [ ]:

X_m1 = m1_df.drop(columns="Target")
y_m1 = m1_df["Target"]

In [ ]:
X_m1_train, X_m1_test, y_m1_train, y_m1_test = model_selection.train_test_split(X_m1, y_m1, random_state=0)

In [ ]:
m1 = linear_model.LinearRegression()
m1.fit(X_m1_train, y_m1_train)
convert_timeunit(metrics.mean_squared_error(y_m1_test, m1.predict(X_m1_test), squared=False), TimeUnit.second, TimeUnit.minute)

In [ ]:
m1_t = tree.DecisionTreeRegressor(max_depth=20)
m1_t.fit(X_m1_train, y_m1_train)
convert_timeunit(metrics.mean_squared_error(y_m1_test, m1_t.predict(X_m1_test), squared=False), TimeUnit.second, TimeUnit.minute)

In [ ]:
degree = 2

polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

polyreg.fit(X_m1_train,y_m1_train)
convert_timeunit(metrics.mean_squared_error(y_m1_test, polyreg.predict(X_m1_test), squared=False), TimeUnit.second, TimeUnit.minute)

In [ ]:
m1_df.head(2)

RMSE prognose

In [ ]:
convert_timeunit(metrics.mean_squared_error(m1_df["Target"], m1_df["Prognose"], squared=False), TimeUnit.second, TimeUnit.minute)

Meer tekst en moet verplaatst worden naar data prep lmao